In [43]:
import json
import requests
import numpy as np
import pandas as pd
import time

In [2]:
df = pd.read_csv('first_week_oct_2015_comments_by_top_400_with_scores_v2.csv')

In [3]:
df.shape

(9970, 16)

In [4]:
df.head()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking,neg,neu,pos,compound,tb_polarity,tb_subjectivity
0,10331981,debacle,debacle,1444055082,2015-10-05 14:24:42+00:00,US is not really scared by BRICS at all. They'...,10331895,NaN,NaN,0,0.000,0.744,0.256,0.7859,0.156250,0.61875
1,10343811,sarciszewski,sarciszewski,1444183995,2015-10-07 02:13:15+00:00,"I wasn't really trying to argue, they said the...",10343761,NaN,NaN,0,0.195,0.805,0.000,-0.3947,0.200000,0.20000
2,10331538,debacle,debacle,1444050490,2015-10-05 13:08:10+00:00,The examples on the homepage kind of underscor...,10331008,NaN,NaN,12,0.000,0.864,0.136,0.2975,0.137500,0.50000
3,10340097,debacle,debacle,1444149186,2015-10-06 16:33:06+00:00,No mention of a critical aspect of a service l...,10339965,NaN,NaN,22,0.214,0.667,0.119,-0.2500,0.000000,0.80000
4,10338552,debacle,debacle,1444136786,2015-10-06 13:06:26+00:00,I think some of these points are gross exagger...,10337763,NaN,NaN,38,0.081,0.902,0.017,-0.8233,0.076667,0.26000


In [16]:
# Add API key here.
api_key = ''

In [17]:
url = ('https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze' +    
    '?key=' + api_key)

In [18]:
data_dict = {
    'comment': {'text': 'what kind of idiot name is foo?'},
    'languages': ['en'],
    'requestedAttributes': {'TOXICITY': {}}
}
response = requests.post(url=url, data=json.dumps(data_dict)) 
response_dict = json.loads(response.content) 
print(json.dumps(response_dict, indent=2))

{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 31,
          "score": {
            "value": 0.9208521,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.9208521,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "en"
  ],
  "detectedLanguages": [
    "en"
  ]
}


In [44]:
def get_toxicity(comment):
    data_dict = {
        'comment': {'text': comment},
        'languages': ['en'],
        'requestedAttributes': {'TOXICITY': {}}
    }
    response = requests.post(url=url, data=json.dumps(data_dict)) 
    response_dict = json.loads(response.content)
    time.sleep(1)
    try:
        return response_dict['attributeScores']['TOXICITY']['summaryScore']['value']
    except:
        return np.nan

In [45]:
get_toxicity(df['text'][17])

0.6780577

In [46]:
df['toxicity'] = df['text'].apply(get_toxicity)

In [56]:
df.index[df['toxicity'].isna()]

Int64Index([19, 39], dtype='int64')

In [59]:
get_toxicity(df['text'][19])

0.062085487

In [66]:
df.at[19, 'toxicity'] = get_toxicity(df['text'][19])

In [68]:
df.iloc[19]

id                                                          10340505
by                                                           debacle
author                                                       debacle
time                                                      1444151736
time_ts                                    2015-10-06 17:15:36+00:00
text               There may be. WordPress is not ironclad and Re...
parent                                                      10340453
deleted                                                          NaN
dead                                                             NaN
ranking                                                            0
neg                                                                0
neu                                                              0.9
pos                                                              0.1
compound                                                      0.3947
tb_polarity                       

In [69]:
df.at[39, 'toxicity'] = get_toxicity(df['text'][39])

In [70]:
df.iloc[39]

id                                                          10343667
by                                                      sarciszewski
author                                                  sarciszewski
time                                                      1444181844
time_ts                                    2015-10-07 01:37:24+00:00
text               > Some implementation principles (least privil...
parent                                                      10343643
deleted                                                          NaN
dead                                                             NaN
ranking                                                            0
neg                                                            0.076
neu                                                            0.723
pos                                                            0.201
compound                                                      0.9489
tb_polarity                       

In [71]:
for text in df['text'][df['toxicity'].isna()]:
    print(text, '\n')

In [79]:
for comment in df.sort_values(by='toxicity', ascending=False)['text'].head(20):
    print(comment, '\n')

>  Fuck you [...] you heartless piece of shit. Please remain civil on HN, even when you feel strongly and someone else feels heartless. https://news.ycombinator.com/newsguidelines.html https://news.ycombinator.com/newswelcome.html 

Of course the math adds up.  You can't run a university without a home in Fire Island, and that shit's spendy. 

We're not on the metric system, man. We are backwards as fuck. 

You know there's a "Black Lives Matter" slogan, right? Feel free to talk about the death of animals - it's an important topic - but linking your campaign the the murder of humans by police officers is, and I say this as politely as I can, fucking stupid. 

Great way to keep the stupid people honest. 

>  These threads always fucking suck on HN. Please don't post things like that to HN. They only make threads suck worse. 

>  Go do some fucking legwork, asshole We've banned this account. You can't do this here. 

what I hate more is the youth of today seem to let this shit happen. No

In [80]:
df.to_csv('first_week_oct_2015_comments_by_top_400_with_scores_v3.csv', index=False)